## Finding frequent pairs: the Pairwise Association Mining problem

Suppose we are a retailer (e.g., Publix, Amazon) who sells items, and we want to discover whether customers buy certain pairs of items together frequently. The data we have are baskets: a basket is the list of items that some customer bought during one of his or her visits. The problem of finding frequent pairs is the pairwise association mining problem.

A simple algorithm could be as follows:
1. Scan all the baskets, and for each basket, initialize and increment the counts for all the pairs in the basket.
2. After going through all the baskets, go through the list of counts and pull out the top occurrences.

Let's build from the ground up; and look at how we can count unique entries in a collection of values. First, we should generate a large collection of random integers.

In [3]:
import random

def generate_rand (n, k):
    return [random.randint (0, n) for _ in range (k)]

random.seed (0xDEADBEEF)
x = generate_rand (1000000000, 1000000)

Using only lists:

In [5]:
def count_unique__using_lists (x):
    """Returns the number of unique entries in `x`."""
    if len(x) == 0: return 0
    assert len(x) >= 1
    y = sorted(x)
    z = [yi != yj for yi, yj in zip(y[:-1], y[1:])]
    return 1 + sum(z)

n_unique = count_unique__using_lists (x)
print ('n =', len (x), 'and n_unique =', n_unique)

n = 1000000 and n_unique = 999492


Now, let us count using only dictionaries. Python has a cool data type called `defaultdict`, which essentially is a dictionary to which we can add values without their respective keys already being present.

In [6]:
from collections import defaultdict
def count_unique__using_dicts (x):
    """Returns the number of unique entries in `x`."""
    if len(x) == 0: return 0
    assert len(x) >= 1
    y = sorted(x)
    z = defaultdict(int)
    for item in y:
        z[item] += 1
    return(len(z))

n_unique = count_unique__using_dicts (x)
print ('n =', len (x), 'and n_unique =', n_unique)

n = 1000000 and n_unique = 999492


Finally, we'll count using a the Python `set` construct.

In [7]:
def count_unique__using_sets (x):
    """Returns the number of unique entries in `x`."""
    return len(set(x))

n_unique = count_unique__using_sets (x)
print ('n =', len (x), 'and n_unique =', n_unique)

n = 1000000 and n_unique = 999492


Okay, we now know how to count the number of unique entries in a collection of numerical values. What about string values though? Let's see how to count (uniquely also) the number of words in a string containing multiple words (a funny example below):

In [9]:
text = """Well, my friend, you are in luck
For an Answer at you I will huck
'Cause when you ask what a Woodchuck will Chuck
You see most folk they don't give a fuck

But I play along like I should
For you want to know just How much wood
And I really will tell you
But I don't want to be misunderstood

When you are driving through the muck
Hoping that you don't get stuck
It is times like these you know you really are a schmuck
Why you want free wood man spend a buck

Keep your foot upon the gas
and eventually you could
Roll up into Woody Woodchuck's 
woodchuck chucking neighborhood"""

In [24]:
words = text.lower().split()
print(len(words))

unique_words = set(words)
print(len(unique_words))

117
83


Now let us write a function that returns a dictionary whose keys are each of the unique letters of the string and whose corresponding values are the counts of each letter. For the moment, we'll discard non-alphabetic characters, e.g., numbers and punctuation.

In [29]:
def count_letters (s):
    """Returns a (default) dictionary of (letter, count) pairs for the given string."""
    counts = defaultdict (int)
    letters = [c for c in s.lower () if c.isalpha ()]
    for letter in letters:
        counts[letter] += 1
    return counts

counts = count_letters (text)
print(counts)

defaultdict(<class 'int'>, {'e': 34, 'd': 22, 'p': 6, 'w': 22, 'l': 27, 'h': 24, 't': 30, 'n': 28, 'a': 27, 's': 18, 'k': 20, 'c': 21, 'm': 8, 'r': 16, 'b': 5, 'j': 1, 'i': 26, 'g': 9, 'f': 7, 'v': 3, 'o': 53, 'u': 38, 'y': 21})


Now, let's assume that each word is a "basket" and let each alphabetic character within a word be an "item" within that basket. Then, if we were to count co-occurring letters in words, we'd in fact be doing pairwise association mining. That is, we're trying to find which pairs of letters tend to co-occur most frequently.

We'll also treat each instance of a repeated word as a distinct basket. Within a basket, we'll count distinct occurrences of each letter. For example, for the word, wood, the pairs (w, o) and (o, d) occur twice each, while the pairs (w, d) and (o, o) occur once each.

Python is cool since it provides handy tool to do just this, i.e., count the number of distinct pairs when given a collection of objects. We can do this as follows:

In [31]:
from itertools import combinations

print (list (combinations ('wood', 2)))

[('w', 'o'), ('w', 'o'), ('w', 'd'), ('o', 'o'), ('o', 'd'), ('o', 'd')]


Now that this is done, let's look at **Sparse Vectors** and **Sparse Matrices**, which are so often used in counting. We call them sparse because we expect most of the values to be 0. We can create them using default dictionaries. Let us use sparse vectors to do what we did before (count the number of occurrences of each alphabet)

In [33]:
def sparse_vector ():
    """Returns an empty sparse vector for storing counts."""
    return defaultdict (int)

def count_letters_spvec (s):
    """Returns a sparse vector of (letter, count) pairs for the given string."""
    counts = sparse_vector ()
    letters = [c for c in s.lower () if c.isalpha ()]
    for letter in letters:
        counts[letter] += 1
    return counts

def print_sparse_vector (x, name=None):
    """Prints a sparse vector with an optional name."""
    if name:
        name += ' '
    else:
        name = ''
    print ("=== Vector {}in Z^{}. ===".format (name, len (x)))
    elements = sorted (x.items (), key=lambda p: p[0])
    for key, value in elements:
        print ("%s: %d" % (key, value))
        
counts_vec = count_letters_spvec (text)
print_sparse_vector (counts_vec, 'counts_vec')

=== Vector counts_vec in Z^23. ===
a: 27
b: 5
c: 21
d: 22
e: 34
f: 7
g: 9
h: 24
i: 26
j: 1
k: 20
l: 27
m: 8
n: 28
o: 53
p: 6
r: 16
s: 18
t: 30
u: 38
v: 3
w: 22
y: 21


Now to the good part! We'll create a sparse matrix using our sparse vector as a building block and use it to count pairs that occur together.

In [38]:
def sparse_matrix ():
    """Returns an empty sparse matrix (2-D table) for storing integer counts."""    
    return defaultdict(sparse_vector)

def count_letter_pairs (s):
    Counts = sparse_matrix ()
    
    baskets = s.lower().split()
    for b in baskets:
        items = [c for c in b if c.isalpha()]
        for i,j in combinations(items,2):
            Counts[i][j] += 1
            if i!=j:
                Counts[j][i] +=1
    
    return Counts

def print_sparse_matrix (X, name=None):
    if not name:
        name = ''
    else:
        name += ' '
        
    nr = len (X)
    nc = max (nr, max ([len (r) for r in X.values ()]))
    
    print ("=== Matrix {}in Z^({}x{}) ===".format (name, nr, nc))
    sorted_rows = sorted (X.items (), key=lambda p: p[0])
    for (i, row_i) in sorted_rows:
        sorted_cols = sorted (row_i.items (), key=lambda p: p[0])
        print ("{:>3s} | {}".format (i, sorted_cols))

X = count_letter_pairs (text)
print_sparse_matrix (X)

=== Matrix in Z^(23x23) ===
  a | [('c', 1), ('d', 4), ('e', 8), ('g', 2), ('h', 2), ('i', 2), ('k', 2), ('l', 8), ('m', 1), ('n', 11), ('o', 1), ('p', 1), ('r', 5), ('s', 5), ('t', 8), ('u', 2), ('v', 1), ('w', 5), ('y', 4)]
  b | [('c', 1), ('d', 1), ('e', 2), ('g', 1), ('h', 2), ('i', 1), ('k', 1), ('n', 1), ('o', 3), ('r', 1), ('t', 2), ('u', 3)]
  c | [('a', 1), ('b', 1), ('c', 6), ('d', 7), ('e', 1), ('f', 1), ('g', 2), ('h', 14), ('i', 2), ('k', 18), ('l', 2), ('m', 4), ('n', 2), ('o', 13), ('s', 6), ('t', 1), ('u', 21), ('w', 6)]
  d | [('a', 4), ('b', 1), ('c', 7), ('d', 1), ('e', 5), ('f', 1), ('g', 2), ('h', 6), ('i', 8), ('k', 4), ('l', 3), ('m', 2), ('n', 12), ('o', 25), ('p', 1), ('r', 5), ('s', 8), ('t', 5), ('u', 8), ('v', 1), ('w', 7), ('y', 1)]
  e | [('a', 8), ('b', 2), ('c', 1), ('d', 5), ('e', 5), ('f', 3), ('g', 3), ('h', 9), ('i', 7), ('k', 4), ('l', 14), ('m', 2), ('n', 9), ('o', 6), ('p', 3), ('r', 11), ('s', 10), ('t', 11), ('u', 5), ('v', 3), ('w', 4), ('y', 

Now let us extract a specific number of 'top pairs' from this mumbo-jumbo of an output, specifically those pairs whose frequency is more than 15

In [44]:
def top_entries (X, s): # s is the frequency threshold
    """Given a sparse (count) matrix, returns a list of
    the pairs that occur at least some number of times.
    """
    top_pairs = []
    items = sorted(X)
    for item in items:
        for item2 in items:
            if X[item][item2]>=s:
                if ((item2, item),X[item2][item]) not in top_pairs:
                    top_pairs.append(((item, item2),X[item][item2]))
    return top_pairs

s = 15
top_pairs = top_entries (X, s)

print ("=== Top entries of X that have frequency more than {} ===".format (s))
for p in top_pairs:
    print (p)

=== Top entries of X that have frequency more than 15 ===
(('c', 'k'), 18)
(('c', 'u'), 21)
(('d', 'o'), 25)
(('h', 'o'), 17)
(('o', 'u'), 26)
(('o', 'w'), 16)
(('o', 'y'), 15)


And now let us work on **actual data**! We'll download some actual shopping basket data and find co-occurring pairs. We can use the shopping basket data available in someone else's [online tutorial](http://www.salemmarafi.com/code/market-basket-analysis-with-r/).

The _Requests_ module makes it easy to download any web page as (raw) text. Below, we'll download the data and store it as the variable `groceries_file`.

In [46]:
import requests
response = requests.get ('http://www.salemmarafi.com/wp-content/uploads/2014/03/groceries.csv')
groceries_file = response.text

In [48]:
def pairwise_assoc_miner (text, s):
    def sparse_vector ():
        return defaultdict (int)
    
    def sparse_matrix ():
        return defaultdict(sparse_vector)
    
    def count_letter_pairs (text):
        #initializing the empty sparse matrix
        Counts = sparse_matrix ()
        
        #splitting the given file by lines so that we get separate baskets
        baskets = text.splitlines()
        
        """this loop goes through all baskets; splits using the comma to get individual items and then forms combinations
        among items of each basket. It then increases counts for both combinations (i,j) and (j,i) while also taking
        care of diagonal elements"""
        for basket in baskets:
            items = basket.split(',')
            for i,j in combinations(items,2):
                Counts[i][j] += 1
                if i!=j:
                    Counts[j][i] +=1
        return Counts
    
    def top_entries (text, s):
        #calling the previous function and storing our sparse matrix in X
        X = count_letter_pairs (text)
        
        #initializing an empty list
        top_pairs = []
        
        #sorting the matrix so that the individual items in baskets are stored in the variable 'items'
        items = sorted(X)
        
        """going through all the pairs of items, and adding to the top_pairs list the pairs whose count is >= s 
        (but only if the mirror pair is not already in the list)"""
        for item in items:
            for item2 in items:
                if X[item][item2]>=s:
                    if ((item2, item),X[item2][item]) not in top_pairs:
                        top_pairs.append(((item, item2),X[item][item2]))
        return top_pairs
    
    #calling the top_entries function on the text and returning it 
    return top_entries(text,s)

And now we can just call the above function, give it a threshold and it'll return the pairs which occur more times than the value of the threshold:

In [50]:
print(pairwise_assoc_miner (groceries_file, 500))

[(('other vegetables', 'whole milk'), 736), (('rolls/buns', 'whole milk'), 557), (('whole milk', 'yogurt'), 551)]


Nice, eh?